# Defining imports:

In [1]:
# !pip install torch-summary

In [2]:
# Standard library imports
import os
import time
from pathlib import Path

# Third-party library imports
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision.utils as vutils
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.io as io
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import TripletMarginLoss
import torch.nn as nn
from torchsummary import summary
from tqdm.notebook import tqdm
from tqdm import tqdm
from typing import Tuple, List, Dict, Union, Any
from pathlib import Path

# Internal imports from PyTorch
from torchvision.models import resnet50
from torchvision.utils import save_image
from torchvision.io import read_image

In [3]:
# # Matplotlib graph settings
# plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
# Check if CUDA (GPU support) is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


# Model:

**Defining custom model: Triplet Neural Network (with dataset class):**

In [5]:
# Utilize PyTorch's data loading utilities
class TripletDataset(torch.utils.data.Dataset):
    
    # Initialization
    def __init__(self, dataset, transform=None):
        
        # Initializes dataset and transformations
        self.dataset = dataset
        self.transform = transform
        
        # Extract label from dataset
        self.labels = [item[1] for item in dataset.imgs]
        
        # Create dictionary where keys are labels and values are lists of indices corresponding to each label.
        self.label_to_indices = {label: np.where(np.array(self.labels) == label)[0]
                                 for label in set(self.labels)}

        
    # Defines how individual items are retrieved from the dataset given an index (Called when dataset is indexed like dataset[index])
    def __getitem__(self, index):
        
        # Extracts the image path and label of the anchor image at the given index from the dataset.
        # label1 will be label of anchor/positive 
        img1, label1 = self.dataset.imgs[index]
        
        # Initialize positive index with the anchor index
        positive_index = index
        
        # For positive index: randomly selects another index from the indices of images with the same label as the anchor image 
        while positive_index == index:
            positive_index = np.random.choice(self.label_to_indices[label1])
            
        # For negative label: Randomly selects a label that is different from the label of the anchor image 
        negative_label = np.random.choice(list(set(self.labels) - set([label1])))
        negative_index = np.random.choice(self.label_to_indices[negative_label])
        
        # Load images corresponding to the anchor, positive, and negative indices and convert images to RGB format
        img2 = self.dataset.imgs[positive_index][0]
        img3 = self.dataset.imgs[negative_index][0]
        img1 = Image.open(img1).convert("RGB")
        img2 = Image.open(img2).convert("RGB")
        img3 = Image.open(img3).convert("RGB")
        
        # If transformation is not None, apply the transformation
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)
        
        # Return images
        return label1, img1, img2, img3

    # Return the length of the dataset
    def __len__(self):
        return len(self.dataset)

In [6]:
class Identity(nn.Module):
    def forward(self, x):
        return x

In [7]:
class TripletNetwork(nn.Module):
    def __init__(self, embedding_size=64):
        super(TripletNetwork, self).__init__()
        
        # Load a pre-trained resnet50 model
        self.backbone = resnet50(pretrained=True)
        
        # Replace the fully connected layer with an Identity module
        self.backbone.fc = Identity()
        
        # Embedding layer
        self.embedding_layer = nn.Linear(2048, embedding_size)  # Use 2048 as the in_features to match ResNet-50
        
        # Classification layers
        self.fc = nn.Sequential(nn.ReLU(), nn.Dropout(0.7), nn.Linear(embedding_size, 1), nn.Sigmoid())

    def forward(self, x, return_embedding=False):
        # Extract features using the backbone
        x = self.backbone(x)  # This will now give a [batch_size, 2048] tensor directly
        
        # Get the embedding
        embedding = self.embedding_layer(x)
        
        if return_embedding:
            return embedding
        
        # Pass embedding through the classification layer
        x = self.fc(embedding)
        return x

# Loading images into Data Loaders:

**Loading images into test data loader:**

In [8]:
class CustomImageFolder(ImageFolder):
    @staticmethod
    def find_classes(directory: Union[str, Path]) -> Tuple[List[str], Dict[str, int]]:
        """
        Finds the class folders in a dataset structured in a directory by overriding the sorting order.

        Parameters:
            directory (Union[str, Path]): Root directory path.

        Returns:
            Tuple[List[str], Dict[str, int]]: (classes, class_to_idx) where classes are a list of 
                                              the class names and class_to_idx is a dictionary mapping 
                                              class name to class index.
        """
        # Correct the syntax error by adding an extra set of parentheses around the generator expression
        classes = sorted((entry.name for entry in os.scandir(directory) if entry.is_dir()), reverse=True)
        if not classes:
            raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

In [9]:
# Set train and valid directory paths
train_directory = 'data/Training_Augmented'
valid_directory = 'data/Validation_Augmented'
test_directory = 'data/Test_Augmented'

# Batch size
bs = 32

# Number of classes
num_classes = 2

#define a standard transform to tensor
transform = transforms.Compose([transforms.ToTensor()])

# Define the index mapping for folders to labels
folder_to_label = {'no_melanoma': 0, 'melanoma': 1}

# Load Data from folders
data = {
    'train': CustomImageFolder(root=train_directory, transform=transform),
    'valid': CustomImageFolder(root=valid_directory, transform=transform),
    'test': CustomImageFolder(root=test_directory, transform=transform)
}

# Modify the class_to_idx attribute to reflect your custom mapping
# data['test'].class_to_idx = folder_to_label

In [10]:
print(data['test'].class_to_idx)

{'no_melanoma': 0, 'melanoma': 1}


In [11]:
train_dataset = TripletDataset(dataset=data['train'], transform=transform)
train_data = DataLoader(train_dataset, batch_size=bs, shuffle=True)

valid_dataset = TripletDataset(dataset=data['valid'], transform=transform)
valid_data = DataLoader(valid_dataset, batch_size=bs, shuffle=True)

test_dataset = TripletDataset(dataset=data['test'], transform=transform)
test_data = DataLoader(test_dataset, batch_size=bs, shuffle=False)

# Initializing the TripletNetwork model:

In [12]:
import torch
import torchvision.models as models

# Instantiate the model
triplet_network = TripletNetwork(embedding_size=64).to(device)

for param in triplet_network.parameters():
    param.requires_grad = False

C:\Users\user\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
summary(triplet_network, input_size=(bs, 9, 408))

Layer (type:depth-idx)                   Param #
├─ResNet: 1-1                            --
|    └─Conv2d: 2-1                       (9,408)
|    └─BatchNorm2d: 2-2                  (128)
|    └─ReLU: 2-3                         --
|    └─MaxPool2d: 2-4                    --
|    └─Sequential: 2-5                   --
|    |    └─Bottleneck: 3-1              (75,008)
|    |    └─Bottleneck: 3-2              (70,400)
|    |    └─Bottleneck: 3-3              (70,400)
|    └─Sequential: 2-6                   --
|    |    └─Bottleneck: 3-4              (379,392)
|    |    └─Bottleneck: 3-5              (280,064)
|    |    └─Bottleneck: 3-6              (280,064)
|    |    └─Bottleneck: 3-7              (280,064)
|    └─Sequential: 2-7                   --
|    |    └─Bottleneck: 3-8              (1,512,448)
|    |    └─Bottleneck: 3-9              (1,117,184)
|    |    └─Bottleneck: 3-10             (1,117,184)
|    |    └─Bottleneck: 3-11             (1,117,184)
|    |    └─Bottleneck: 3

Layer (type:depth-idx)                   Param #
├─ResNet: 1-1                            --
|    └─Conv2d: 2-1                       (9,408)
|    └─BatchNorm2d: 2-2                  (128)
|    └─ReLU: 2-3                         --
|    └─MaxPool2d: 2-4                    --
|    └─Sequential: 2-5                   --
|    |    └─Bottleneck: 3-1              (75,008)
|    |    └─Bottleneck: 3-2              (70,400)
|    |    └─Bottleneck: 3-3              (70,400)
|    └─Sequential: 2-6                   --
|    |    └─Bottleneck: 3-4              (379,392)
|    |    └─Bottleneck: 3-5              (280,064)
|    |    └─Bottleneck: 3-6              (280,064)
|    |    └─Bottleneck: 3-7              (280,064)
|    └─Sequential: 2-7                   --
|    |    └─Bottleneck: 3-8              (1,512,448)
|    |    └─Bottleneck: 3-9              (1,117,184)
|    |    └─Bottleneck: 3-10             (1,117,184)
|    |    └─Bottleneck: 3-11             (1,117,184)
|    |    └─Bottleneck: 3

In [14]:
# Unfreeze the parameters of the last residual block and the embedding layer
for param in triplet_network.backbone.layer4.parameters():
    param.requires_grad = True

In [15]:
summary(triplet_network, input_size=(bs, 9, 408))

Layer (type:depth-idx)                   Param #
├─ResNet: 1-1                            --
|    └─Conv2d: 2-1                       (9,408)
|    └─BatchNorm2d: 2-2                  (128)
|    └─ReLU: 2-3                         --
|    └─MaxPool2d: 2-4                    --
|    └─Sequential: 2-5                   --
|    |    └─Bottleneck: 3-1              (75,008)
|    |    └─Bottleneck: 3-2              (70,400)
|    |    └─Bottleneck: 3-3              (70,400)
|    └─Sequential: 2-6                   --
|    |    └─Bottleneck: 3-4              (379,392)
|    |    └─Bottleneck: 3-5              (280,064)
|    |    └─Bottleneck: 3-6              (280,064)
|    |    └─Bottleneck: 3-7              (280,064)
|    └─Sequential: 2-7                   --
|    |    └─Bottleneck: 3-8              (1,512,448)
|    |    └─Bottleneck: 3-9              (1,117,184)
|    |    └─Bottleneck: 3-10             (1,117,184)
|    |    └─Bottleneck: 3-11             (1,117,184)
|    |    └─Bottleneck: 3

Layer (type:depth-idx)                   Param #
├─ResNet: 1-1                            --
|    └─Conv2d: 2-1                       (9,408)
|    └─BatchNorm2d: 2-2                  (128)
|    └─ReLU: 2-3                         --
|    └─MaxPool2d: 2-4                    --
|    └─Sequential: 2-5                   --
|    |    └─Bottleneck: 3-1              (75,008)
|    |    └─Bottleneck: 3-2              (70,400)
|    |    └─Bottleneck: 3-3              (70,400)
|    └─Sequential: 2-6                   --
|    |    └─Bottleneck: 3-4              (379,392)
|    |    └─Bottleneck: 3-5              (280,064)
|    |    └─Bottleneck: 3-6              (280,064)
|    |    └─Bottleneck: 3-7              (280,064)
|    └─Sequential: 2-7                   --
|    |    └─Bottleneck: 3-8              (1,512,448)
|    |    └─Bottleneck: 3-9              (1,117,184)
|    |    └─Bottleneck: 3-10             (1,117,184)
|    |    └─Bottleneck: 3-11             (1,117,184)
|    |    └─Bottleneck: 3

In [16]:
# Assuming you replaced the fully connected layer with an embedding layer
for param in triplet_network.backbone.fc.parameters():
    param.requires_grad = True

# Training & Validating TripletNetwork model:

In [ ]:
epochs = 10
patience = 3  # For early stopping
best_valid_loss = float('inf')
early_stopping_counter = 0

# Define the path where you want to save the model
model_path = "model_weights.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
triplet_network.to(device)
optimizer = Adam(triplet_network.parameters(), lr=0.001,weight_decay=1e-2)
loss_func = TripletMarginLoss(margin=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.99)

history = []

for epoch in tqdm(range(epochs)):
    epoch_start = time.time()
    train_loss, valid_loss = 0.0, 0.0
    correct_train, correct_valid = 0, 0
    total_train, total_valid = 0, 0
    
    triplet_network.train()
    for labels, anchor, positive, negative in train_data:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        
        optimizer.zero_grad()
        anchor_embed = triplet_network(anchor)
        positive_embed = triplet_network(positive)
        negative_embed = triplet_network(negative)
        
        loss = loss_func(anchor_embed, positive_embed, negative_embed)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        # Calculate "accuracy"
        with torch.no_grad():
            dist_positive = (anchor_embed - positive_embed).pow(2).sum(1)  # Euclidean distance
            dist_negative = (anchor_embed - negative_embed).pow(2).sum(1)
            correct_train += (dist_positive < dist_negative).sum().item()
            total_train += anchor.size(0)
    
    scheduler.step()
    triplet_network.eval()
    with torch.no_grad():
        for labels, anchor, positive, negative in valid_data:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
            anchor_embed = triplet_network(anchor)
            positive_embed = triplet_network(positive)
            negative_embed = triplet_network(negative)
            
            loss = loss_func(anchor_embed, positive_embed, negative_embed)
            valid_loss += loss.item()
            
            # Calculate "accuracy"
            dist_positive = (anchor_embed - positive_embed).pow(2).sum(1)
            dist_negative = (anchor_embed - negative_embed).pow(2).sum(1)
            correct_valid += (dist_positive < dist_negative).sum().item()
            total_valid += anchor.size(0)
    
    avg_train_loss = train_loss / len(train_data)
    avg_valid_loss = valid_loss / len(valid_data)
    train_accuracy = correct_train / total_train
    valid_accuracy = correct_valid / total_valid
    
    history.append([avg_train_loss, avg_valid_loss, train_accuracy, valid_accuracy])
    
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        early_stopping_counter = 0
        # Save the model's weights
        torch.save(triplet_network.state_dict(), model_path)
        
    else:
        early_stopping_counter += 1
        if early_stopping_counter > patience:
            print("Early stopping triggered.")
            break

    epoch_end = time.time()
    print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Valid Loss: {avg_valid_loss:.4f}, Valid Acc: {valid_accuracy:.4f}, Time: {epoch_end - epoch_start:.2f}s")

# Computing Reference Embeddings:

In [ ]:
valid_dataset_reference = CustomImageFolder(root=valid_directory, transform=transform)
valid_data_reference = DataLoader(valid_dataset_reference, batch_size=bs, shuffle=False)

In [ ]:
def compute_reference_embeddings(model, dataloader, device):
    model.eval()
    embeddings = []
    labels = []

    with torch.no_grad():
        for images, label in dataloader:
            images = images.to(device)
            output = model(images)  # Ensure this gets the embedding before the sigmoid
            # Adjust here to append each embedding in the batch individually
            embeddings.extend(output.cpu().detach().numpy())
            labels.extend(label.cpu().numpy())  # Assuming labels are not already on CPU and not converted to list

    # Convert embeddings back to tensor after collecting them
    embeddings = torch.tensor(embeddings, dtype=torch.float32, device=device)

    # Average the embeddings for each class
    class_embeddings = {}
    unique_labels = set(labels)
    for lbl in unique_labels:
        class_indices = [i for i, x in enumerate(labels) if x == lbl]
        class_embeddings[lbl] = torch.mean(embeddings[class_indices], dim=0)

    return class_embeddings

In [21]:
# Initiliaze the Triplet Network 
loaded_model = TripletNetwork(embedding_size=64) 

# Load the model from model_path
#loaded_model.load_state_dict(torch.load("model_weights_3_epoch_Triplet.pth"))
loaded_model.load_state_dict(torch.load("model_weights.pth"))

# Set it in eval mode
#loaded_model.eval()

# Move model to device
loaded_model = loaded_model.to(device)  

In [20]:
reference_embeddings = compute_reference_embeddings(loaded_model, valid_data_reference, device)

# Save the reference embeddings for later use
torch.save(reference_embeddings, 'reference_embeddings.pt')

NameError: name 'compute_reference_embeddings' is not defined

# Predict One Image:

In [22]:
def predict_class(model, image, reference_embeddings, device):
    
    # Switch model to evaluation mode
    model.eval()  
    
    
    with torch.no_grad():
        # Make sure the image has a batch dimension
        if image.dim() == 3:
            image = image.unsqueeze(0)  # Add batch dimension if not present
        image = image.to(device)
        
        print("Input shape to model:", image.shape)
        
        # Get the embedding of the uploaded image
        image_embedding = model(image)

    # Initialize the closest class and smallest distance
    closest_class = None
    smallest_distance = float('inf')

    # Compare the uploaded image's embedding to each reference embedding
    for class_name, ref_embedding in reference_embeddings.items():
        distance = (image_embedding - ref_embedding.to(device)).pow(2).sum(1).item()
        if distance < smallest_distance:
            smallest_distance = distance
            closest_class = class_name

    return closest_class

In [23]:
def load_image(image_path, transform, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)  # Add batch dimension and send to device
    return image

# Define the transform
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Example image path
image_path = "data/Test_Augmented/melanoma/ISIC_0035914_v1.jpg"

# Load the image
image = load_image(image_path, transform, device)

# Load embeddings
reference_embeddings = torch.load('reference_embeddings.pt')

# Now you can directly use the loaded image for prediction
predicted_class = predict_class(loaded_model, image, reference_embeddings, device)

Input shape to model: torch.Size([1, 3, 224, 224])


In [24]:
print(predicted_class)

1


# Predict Multiple Images:

In [43]:
class CustomImageFolderPrediction(ImageFolder):
    @staticmethod
    def find_classes(directory: Union[str, os.PathLike]) -> Tuple[List[str], Dict[str, int]]:
        """
        Finds the class folders in a dataset structured in a directory by overriding the sorting order.

        Parameters:
            directory (Union[str, os.PathLike]): Root directory path.

        Returns:
            Tuple[List[str], Dict[str, int]]: (classes, class_to_idx) where classes are a list of 
                                              the class names sorted in reverse alphabetical order, 
                                              and class_to_idx is a dictionary mapping class name to class index.
        """
        classes = sorted([entry.name for entry in os.scandir(directory) if entry.is_dir()], reverse=True)
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

    def __getitem__(self, index: int):
        """
        Override the __getitem__ method to return the path along with the image and label.

        Parameters:
            index (int): Index of the item.

        Returns:
            tuple: (image, label, path) where image is the transformed image tensor, label is the class label of the image, 
                   and path is the file path of the image.
        """
        # Call the original __getitem__ to get the image and label
        original_tuple = super(CustomImageFolderPrediction, self).__getitem__(index)
        # Get the image path
        path = self.imgs[index][0]  # self.imgs is a list of (image path, class index) tuples
        # Return a new tuple that includes the original content plus the path
        return original_tuple + (path,)

In [44]:
# Initiliaze the Triplet Network 
loaded_model = TripletNetwork(embedding_size=64) 

# Load the model from model_path
#loaded_model.load_state_dict(torch.load("model_weights_3_epoch_Triplet.pth"))
loaded_model.load_state_dict(torch.load("model_weights.pth"))

# Set it in eval mode
#loaded_model.eval()

# Move model to device
loaded_model = loaded_model.to(device)  

In [45]:
test_dataset_predict = CustomImageFolderPrediction(root = test_directory, transform=transform)
test_dataloader_predict = DataLoader(test_dataset_predict, batch_size=bs, shuffle=False)

# Load embeddings
reference_embeddings = torch.load('reference_embeddings.pt')

In [53]:
def predict_classes(model, data_loader, reference_embeddings, device):
    model.eval() 
    predictions = {}

    with torch.no_grad():
        for images, _, paths in data_loader:  # Assuming paths are returned by the DataLoader
            images = images.to(device)
            image_embeddings = model(images)

            for i in range(images.size(0)):
                smallest_distance = float('inf')
                closest_class = None
                
                for class_name, ref_embedding in reference_embeddings.items():
                    distance = (image_embeddings[i] - ref_embedding.to(device)).pow(2).sum().item()
                    if distance < smallest_distance:
                        smallest_distance = distance
                        closest_class = class_name
                
                filename = os.path.basename(paths[i])
                predictions[filename] = closest_class

    return predictions

In [54]:
predicted_classes = predict_classes(loaded_model, test_dataloader_predict, reference_embeddings, device)

In [55]:
print(predicted_classes)

{'ISIC_0034529_v1.jpg': 1, 'ISIC_0034548_v1.jpg': 1, 'ISIC_0034572_v1.jpg': 1, 'ISIC_0034573_v1.jpg': 1, 'ISIC_0034584_v1.jpg': 0, 'ISIC_0034595_v1.jpg': 1, 'ISIC_0034605_v1.jpg': 0, 'ISIC_0034628_v1.jpg': 1, 'ISIC_0034630_v1.jpg': 0, 'ISIC_0034633_v1.jpg': 1, 'ISIC_0034638_v1.jpg': 1, 'ISIC_0034644_v1.jpg': 1, 'ISIC_0034650_v1.jpg': 1, 'ISIC_0034655_v1.jpg': 1, 'ISIC_0034657_v1.jpg': 1, 'ISIC_0034687_v1.jpg': 1, 'ISIC_0034713_v1.jpg': 1, 'ISIC_0034714_v1.jpg': 1, 'ISIC_0034737_v1.jpg': 1, 'ISIC_0034764_v1.jpg': 1, 'ISIC_0034771_v1.jpg': 1, 'ISIC_0034784_v1.jpg': 0, 'ISIC_0034804_v1.jpg': 1, 'ISIC_0034817_v1.jpg': 0, 'ISIC_0034819_v1.jpg': 1, 'ISIC_0034859_v1.jpg': 1, 'ISIC_0034863_v1.jpg': 1, 'ISIC_0034871_v1.jpg': 1, 'ISIC_0034873_v1.jpg': 0, 'ISIC_0034874_v1.jpg': 1, 'ISIC_0034875_v1.jpg': 0, 'ISIC_0034882_v1.jpg': 1, 'ISIC_0034895_v1.jpg': 1, 'ISIC_0034904_v1.jpg': 1, 'ISIC_0034918_v1.jpg': 1, 'ISIC_0034926_v1.jpg': 0, 'ISIC_0034934_v1.jpg': 1, 'ISIC_0034949_v1.jpg': 1, 'ISIC_00349

In [68]:
# Predicted class dataframe
predicted_classes_df = pd.DataFrame(list(predicted_classes.items()), columns=['Image', 'Predicted Value'])

# Remove '_v1' from the 'Filename' column
predicted_classes_df['Image'] = predicted_classes_df['Image'].str.replace('_v1', '')

# Display the DataFrame
display(predicted_classes_df)

,Image,Predicted Value
0,ISIC_0034529.jpg,1
1,ISIC_0034548.jpg,1
2,ISIC_0034572.jpg,1
3,ISIC_0034573.jpg,1
4,ISIC_0034584.jpg,0
...,...,...
1507,ISIC_0036060.jpg,0
1508,ISIC_0036061.jpg,0
1509,ISIC_0036062.jpg,0
1510,ISIC_0036063.jpg,1


**Loading ground truth:**

In [70]:
# Read the CSV file
ground_truth_df = pd.read_csv("ISIC2018_Task3_Test_GroundTruth.csv", dtype={"MEL": int})

# Rename the first column to "Image"
ground_truth_df = ground_truth_df.rename(columns={ground_truth_df.columns[0]: "Image"})

# Add '.jpg' to every value in the 'image' column
ground_truth_df['Image'] = ground_truth_df['Image'] + '.jpg'

# Extract the first two columns and rename the second column
ground_truth_df = ground_truth_df.iloc[:, :2]  # Extracting first two columns
ground_truth_df = ground_truth_df.rename(columns={"MEL": "Ground truth labels"})  # Renaming the second column

# Display the DataFrame
display(ground_truth_df)

,Image,Ground truth labels
0,ISIC_0034524.jpg,0
1,ISIC_0034525.jpg,0
2,ISIC_0034526.jpg,0
3,ISIC_0034527.jpg,0
4,ISIC_0034528.jpg,0
...,...,...
1507,ISIC_0036060.jpg,0
1508,ISIC_0036061.jpg,0
1509,ISIC_0036062.jpg,0
1510,ISIC_0036063.jpg,0


**Merged dataframe:**

In [72]:
# Perform inner join on "Image" column
merged_df = pd.merge(predicted_classes_df, ground_truth_df, on="Image")

# Display the merged DataFrame
display(merged_df)

,Image,Predicted Value,Ground truth labels
0,ISIC_0034529.jpg,1,1
1,ISIC_0034548.jpg,1,1
2,ISIC_0034572.jpg,1,1
3,ISIC_0034573.jpg,1,1
4,ISIC_0034584.jpg,0,1
...,...,...,...
1507,ISIC_0036060.jpg,0,0
1508,ISIC_0036061.jpg,0,0
1509,ISIC_0036062.jpg,0,0
1510,ISIC_0036063.jpg,1,0


In [76]:
# Find rows where the Predicted Value is different from the Ground truth labels
different_rows = merged_df[merged_df['Predicted Value'] != merged_df['Ground truth labels']]

# Find rows where the Predicted Value is the same as the Ground truth labels
same_rows = merged_df[merged_df['Predicted Value'] == merged_df['Ground truth labels']]

In [77]:
# Display the different rows
display(different_rows)

,Image,Predicted Value,Ground truth labels
4,ISIC_0034584.jpg,0,1
6,ISIC_0034605.jpg,0,1
8,ISIC_0034630.jpg,0,1
21,ISIC_0034784.jpg,0,1
23,ISIC_0034817.jpg,0,1
...,...,...,...
1499,ISIC_0036049.jpg,1,0
1504,ISIC_0036055.jpg,1,0
1505,ISIC_0036058.jpg,1,0
1510,ISIC_0036063.jpg,1,0


In [78]:
# Display the same rows
display(same_rows)

,Image,Predicted Value,Ground truth labels
0,ISIC_0034529.jpg,1,1
1,ISIC_0034548.jpg,1,1
2,ISIC_0034572.jpg,1,1
3,ISIC_0034573.jpg,1,1
5,ISIC_0034595.jpg,1,1
...,...,...,...
1503,ISIC_0036054.jpg,0,0
1506,ISIC_0036059.jpg,0,0
1507,ISIC_0036060.jpg,0,0
1508,ISIC_0036061.jpg,0,0


# Loading & conducting inferencing:

In [ ]:
def extract_unique_value(tensor):
    
    # If all elements in the tensor evaluate to True, return 1
    if tensor.all():
        return 1
    
    # If none of the elements evaluate to True, return 0
    elif not tensor.any():
        return 0
    
    # Else, return None (contains mix of 0 and 1)
    else:
        return None

In [ ]:
def test_model(model, test_loader, device):
    """
    Test the trained model on the test dataset and compute performance metrics.
    
    Args:
        model: The trained PyTorch model to evaluate.
        test_loader: DataLoader for the test dataset.
        device: The device to use for inference (default: cpu).
    Returns:
        Performance metrics such as accuracy, precision, recall, etc.
    """
    
    # Initialize predicted_label
    predicted_label = None

    # Don't include gradient in testing
    with torch.no_grad():
        
        
        for label, anchor, positive, negative in test_loader:
            
            # Send anchor, positive and negative to device
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
            
            # Obtain ground_truth from label tensor
            ground_truth = extract_unique_value(label)
            
            # Generate embeddings for anchor, positive and negative through the model
            anchor_embed = model(anchor)
            positive_embed = model(positive)
            negative_embed = model(negative)
            
            # Calculate squared Euclidean distance between the anchor and positive/negative embeddings
            dist_positive = (anchor_embed - positive_embed).pow(2).sum(1)
            dist_negative = (anchor_embed - negative_embed).pow(2).sum(1)
            
            # If distance between anchor & positive embedding is smaller than that of anchor & negative embedding, consider it as correct prediction
            if (dist_positive < dist_negative).sum().item():
                
                print("Correct prediction made by model \n")
                
                if ground_truth == 1:
                    
                    # Set predicted_label same as ground_truth
                    predicted_label = ground_truth 
                    
                    print("Ground truth for the image: Melanoma")
                    print(f"Model's prediction for the image: Melanoma \n")
                    
                else:
                    
                    # Set predicted_label same as ground_truth
                    predicted_label = ground_truth 
                    
                    print("Ground truth for this particular image was: Not Melanoma")
                    print(f"Model's prediction for the image: Not Melanoma \n")
                    
            else:
                
                print("Incorrect prediction made by model \n")
                
                
                if ground_truth == 1:
                    
                    # Set predicted_label opposite as ground_truth
                    predicted_label = 0 
                    
                    print("Ground truth for the image: Melanoma")
                    print(f"Model's prediction for the image: Not Melanoma \n")
                    
                else:
                    
                    # Set predicted_label opposite as ground_truth
                    predicted_label = 1 
                    
                    print("Ground truth for the image: Not Melanoma")
                    print(f"Model's prediction for the image: Melanoma \n")
    
    return ground_truth, predicted_label

In [ ]:
ground_truth, predicted_label = test_model(loaded_model, test_data, device)

In [ ]:
print(ground_truth)
print(predicted_label)